# Introduction

We're going to look at the cleaning, wrangling and exploratory data analysis (EDA) process using an example dataset on 
[adult income](https://www.kaggle.com/datasets/uciml/adult-census-income/data) taken from a sample of census data in the United States.

The original purpose of the data for for a competition on the [Kaggle](https://www.kaggle.com) website to try to use machine learning to predict whether an individual would have an income of more than, or less than $50k/year. This is an example of a 'classification' problem.

For our purposes, we just want to explore using basic Python programming to tidy-up and expore the data set.

## Load Libraries

At the start of most Python code, it's good practice to load in the additional libraries we're going to need. Here we're going to use:
* `pandas`, which allows manipulation of data in structures called 'dataframes', which act a bit like Excel spreadsheets.
* `Path`, which makes navigatin around file paths and folders a bit easier.
* `numpy`, which is used for scientific computing in Python, enabling manipulation of arrays of numbers.

In [ ]:
import pandas
from pathlib import Path
import numpy as np

## Setup

After loading our required libraries, it's probably a good idea to tell our notebook where it can find the data we want to use. Don't worry too much about this code, it just sets up the correct path to the data on your machine. 

In [ ]:
user = "user-name"
#root_path = Path(f"/home/jupyter-{user}/bootcamp") 
root_path = Path('../')
data_name = "income"

## Read Data

Next we need to read in our data. `pandas` provides functions for reading data in from a variety of file types, including Microsoft Excel. For this excercise we'll, use a '.csv' (comma separated variable) file.

In [ ]:
raw_data = pandas.read_csv(root_path / f"data/raw/{data_name}.csv")

## View Data

You can look at all of the data in a pandas dataframe by just using the dataframe's variable name in code - e.g. here by using `raw_data`. This might not be a great thing to do if the dataframe has a significant number of rows or columns.

You can view the top 'n' rows using the Pandas `pead` function, or the bottom 'n' rows using the `tail` function, and providing a number for 'n' as a parameter within the function brackets. If you don't provide a number, pandas will default 'n' to a value of 5. 

**NB: Viewing the data isn't strictly necessary as a cleaning step. Once set up correctly a cleaning script could run happi;y without any human intervention or analysis (and without being run as individual cells in a notebook). We'll allow some analysis here for educational purposes.**

In [ ]:
raw_data.head()

If we want a bit of information about the columns in our dataframe, their data types and the number of missing values we have, we can use the `info` function. Note here `info` reports two data types ('Dtypes') - `int64` or integer numbers, and `object` which probably means the column contains text (also called 'string') information.

In [ ]:
raw_data.info()

if we want a quick summary of the statistical properties of the numeric data columns, then we can use the `describe` function.

In [ ]:
raw_data.describe()

## Clean Data

### Format Column Names

Let's tidy up the column names to consistently use underscores between words, and expand some of the names so that their meaning is clearer. We can do this via the `rename` function, and providing a Python dictionary called `columns` of the column names we need to change. The dictionary consists of a list of `key : value` pairs, enclosed in curly braces `{}`. The `key` is the original column name we're looking-up, the `value` is what we want to change the name to.

In [ ]:
raw_data = raw_data.rename(columns={
    'fnlwgt': 'final_weight',
    'educational-num': 'education_code',
    'marital-status': 'marital_status',
    'capital-gain': 'capital_gain',
    'capital-loss': 'capital_loss',
    'hours-per-week': 'hours_per_week',
    'native-country': "native_country",
})

### Format Missing Values

Suppose we had missing values in our dataset, and suppose those missing values were indicated by a `'?'` character. Python wouldn't initially understand that those values are intended to be missing - so we'd need to replace them by something it does understand. Here we use the `replace` function to scan the entire dataframe and replace any `'?'` encountered with a special value `NaN` (Not-a-Number) that Python does understand as indicationg missing data.

In [ ]:
raw_data = raw_data.replace({
    '?': np.NaN,
})

### Format Column Types

As noted previously, our numeric columns have been interpreted by `pandas` as holding integer (whole number) values. This can lead to some oddities in maths calculations (e.g. thinking about rounding, if we're only using whole numbers, what is (3+2)/2?). We'll therefore use the `astype` function to reinterpret our numeric columns as floating point values. 

In [ ]:
raw_data = raw_data.astype({
    'age': 'float',
    'final_weight': 'float',
    'capital_gain': 'float',
    'capital_loss': 'float',
    'hours_per_week': 'float',
})

### Test for Uniquness

Perhaps there are some issues in the data, such as duplicated rows that could affect our analyses and distort our results? We can use the `duplicated` function to return duplicated rows, but suppose we just want to know the number of rows duplicated? We can 'chain' another function, `sum` to take the results from calling `duplicated` to add up the number of repeated rows.

If duplicated rows are found, then we can use `drop_duplicates` to remove them from our dataset. The third and fourth lines of code are an example of a 'conditio9nal statement' - *if* a condition is met, *then* do something. Note that the conditional statement ends in a `':'` character, and the action to be taken if the conditional evaluates to `'True'` is indented on the next line. Indentation is very important in Python so that it knows which 'blocks' of code are meant ot be run consecutively.  

In [ ]:
n_duplicates = raw_data.duplicated().sum()
print(f'There are {n_duplicates} duplicates.')
if n_duplicates > 0: 
    raw_data = raw_data.drop_duplicates()

### Test for Validity

Another common data cleaning check is for the validity of data. For example we don't expect anyone to have a negative value for their age, nor do we expect anyone to be over 120 years old. Let's count up these two conditions and check to see if there are any occurences. If there are, we, as data scientists, have a choice to make - do we ignore these outliers? try to replace them with more sensible values through imputation? or just remove these data from the dataset? 

In this instance, we'll do the latter. Note that the `'|'` character means 'or' (and the `'&'` means 'and') and links the two age expressions together. 

In [ ]:
condition = (raw_data['age'] < 0) | (raw_data['age'] > 120)
n_age_outliers = condition.sum()
print(f'There are {n_age_outliers} age outliers.')
if n_age_outliers > 0:
    raw_data = raw_data.drop(raw_data[condition].index)

### Test for Consistency

Occasionally our data may contain multiple columns that should contain the same information, just encoded in a different way - how can we check that these collumns have the correct correspondence and are logically consistent? (e.g. a questionnaire survey may have three responses: 'yes', 'no', 'maybe' to a particular question, for analysis, these should be encoded '1', '0', '2' in another column, but, if that encoding has been done manually, has it been done correctly?)

If we want to look at the unique values in a particular column, we can use the `unique` function. Let's quickly look at the the values associated with the 'education' column.

In [ ]:
raw_data['education'].unique()

Of more interest (probably) is the count of the number of rows associated with the uniques values. For this we can use the `value_counts` function.

In [ ]:
raw_data['education'].value_counts()

We also note in the data that there is an 'education_code' column. What does this contain? Intuitively, you would expect a code that corresponds to the text in the 'education' column? Let's check...

In [ ]:
raw_data['education_code'].value_counts()

At first glance, it does appear that the 'education_code' does correctly match the 'education' text. Let's quickly write some code to check that the two sets of value counts for 'education' and 'education_code' match correctly. 

In [ ]:
n_education_errors = (raw_data['education'].value_counts().values != raw_data['education_code'].value_counts().values).sum()
print(f"There are {n_education_errors} education errors.")

## Write Data

Finally, we probably want to save our cleaned data off into another location so that we don't need to clean the raw data again. We still keep the raw data around of course, so that we can go back to it if necessary. 

The code below also demonstrates saving the data off as another type of file - a 'pickle' file. Unlike the '.csv' file we loaded the raw data in from, the pickle file is native to Python, and preserves the changes we've made (e.g. turning integer columns into floats). If we just saved our clean data as a '.csv' again, we would need to repeat that data type conversion every time we loaded our data.

In [ ]:
raw_data.to_pickle(root_path / f"data/clean/{data_name}.pkl")